In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import math
import tqdm
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from datetime import *
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

from sklearn.ensemble import RandomForestRegressor
from scipy.stats import skew, kurtosis, hmean, gmean
import itertools

import autoreload
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [488]:
import data_provider

In [489]:
df = data_provider.df

In [ ]:
pred_size = data_provider.pred_size
hist_window = 43
# scaler = MinMaxScaler() #15562vs 15k w/o
# scaler_y = MinMaxScaler(feature_range=(0, 1))

In [564]:
def split_train_val(df, hist_window, pred_size):
    dataForModel = df[(df["Date"] > df["Date"].max() - timedelta(days=pred_size*2 + hist_window))]
    dfModel = pd.DataFrame()
    dfVal = pd.DataFrame()
    for pid in tqdm.tqdm(sorted(dataForModel.player_id.unique())):
        s=data_provider.series_to_supervised(dataForModel[dataForModel.player_id == pid]['price'].tolist(), n_in=hist_window, n_out=pred_size, dropnan=True)
        dates =dataForModel[dataForModel.player_id == pid]['Date'].shift(pred_size).iloc[-(pred_size+1):]
#         print(s.shape, dates.shape)
        dates.index=s.index
        s = pd.concat([dates,s],axis=1)
        stat = data_provider.df_stats[data_provider.df_stats['player_id']==pid]
        stat_model= pd.DataFrame()
        for i in range(s.shape[0]-1):
            stat_model = pd.concat([stat_model, stat])
        stat_model.index = s.iloc[:-1, :].index
        dfModel = pd.concat([dfModel, stat_model.join(s.iloc[:-1, :])])
        stat.index = s.iloc[-1:, :].index
        dfVal = pd.concat([dfVal,  stat.join(s.iloc[-1:, :])])
    print("Data is ready!!!")
    print(dfModel.shape)
    print(dfVal.shape)
    return dfModel, dfVal

def scale_split_train_test_val(dfModel , dfVal, hist_window, pred_size):
    stat_size = dfModel.shape[1]-hist_window-pred_size
    print(stat_size)
    X = dfModel.iloc[:, :stat_size+hist_window]
    X_val = dfVal.iloc[:, :stat_size+hist_window]

    y = dfModel.iloc[:, -pred_size:]
    
    x_cols=X.columns
    y_cols=y.columns
    
#     X = pd.DataFrame(scaler.fit_transform(X), columns=x_cols)
#     y = pd.DataFrame(scaler_y.fit_transform(y), columns=y_cols)

    X_val = dfVal.iloc[:, :stat_size+hist_window]
#     X_val = pd.DataFrame(scaler.transform(X_val), columns=x_cols)

    y_val = dfVal.iloc[:, -pred_size:]
#     y_val = pd.DataFrame(scaler_y.transform(y_val), columns=y_cols)
    X_test = pd.concat([dfVal.iloc[:, :stat_size], dfVal.iloc[:, -hist_window:]], axis=1)
    X_test.columns=x_cols
    
#     X_test = pd.DataFrame(scaler.transform(X_test), columns=x_cols)
    return X, y, X_val, y_val, X_test
    
def unscale_y(y):
    return scaler_y.inverse_transform(y)


def root_mean_squared_error(y_true, y_pred):
        return math.sqrt(mean_squared_error(y_pred, y_true))
    
def evaluate(y_true, y_pred):
    return root_mean_squared_error(np.exp(y_true), np.exp(y_pred))

In [565]:
df_train, df_val = split_train_val(df, hist_window, pred_size)

100%|██████████| 1253/1253 [00:26<00:00, 47.84it/s]

Data is ready!!!
(8771, 55)
(1253, 55)


In [375]:
df_train.columns

Index(['colour', 'rareness', 'player_id', 'revision', 'Date', 'var1(t-43)',
       'var1(t-42)', 'var1(t-41)', 'var1(t-40)', 'var1(t-39)', 'var1(t-38)',
       'var1(t-37)', 'var1(t-36)', 'var1(t-35)', 'var1(t-34)', 'var1(t-33)',
       'var1(t-32)', 'var1(t-31)', 'var1(t-30)', 'var1(t-29)', 'var1(t-28)',
       'var1(t-27)', 'var1(t-26)', 'var1(t-25)', 'var1(t-24)', 'var1(t-23)',
       'var1(t-22)', 'var1(t-21)', 'var1(t-20)', 'var1(t-19)', 'var1(t-18)',
       'var1(t-17)', 'var1(t-16)', 'var1(t-15)', 'var1(t-14)', 'var1(t-13)',
       'var1(t-12)', 'var1(t-11)', 'var1(t-10)', 'var1(t-9)', 'var1(t-8)',
       'var1(t-7)', 'var1(t-6)', 'var1(t-5)', 'var1(t-4)', 'var1(t-3)',
       'var1(t-2)', 'var1(t-1)', 'var1(t)', 'var1(t+1)', 'var1(t+2)',
       'var1(t+3)', 'var1(t+4)', 'var1(t+5)', 'var1(t+6)'],
      dtype='object')

In [566]:
min_date = df_train.Date.min()
df_train = df_train[df_train.Date == min_date]

In [557]:
### test on lr -> output/stat_feature_result.json -> worse with 'weekday', 'is_cheap'
# col = ['weekday', 'is_cheap'] + df_train.columns.values.tolist() 
# df_train['weekday'] = df_train['Date'].apply(lambda x: x.weekday())
# df_train['is_cheap'] = np.exp(df_train['var1(t)'])<50000
# df_train['is_cheap'] = df_train['is_cheap'].astype(np.int8)
# df_train = df_train[col]

# df_val['weekday'] = df_val['Date'].apply(lambda x: x.weekday())
# df_val['is_cheap'] = np.exp(df_val['var1(t)'])<50000
# df_val['is_cheap'] = df_val['is_cheap'].astype(np.int8)
# df_val = df_val[col]

In [567]:
test_col = ['var1(t)', 'var1(t+1)','var1(t+2)', 'var1(t+3)', 'var1(t+4)', 'var1(t+5)', 'var1(t+6)']

train_col = []
for col in df_train.columns:
        if 'var1' in col:
            train_col.append(col)

train_col = list(set(train_col).difference(test_col))
train_col = sorted(train_col, key=lambda x : int(x.split('t')[1].split(')')[0]))

df_train.drop(['Date', 'player_id'], inplace=True, axis=1)
df_val.drop(['Date', 'player_id'], inplace=True, axis=1)

In [568]:
X, y, X_val, y_val, X_test = scale_split_train_test_val(df_train, df_val, hist_window, pred_size)

3


In [580]:
add_cols = []
for c in X.columns:
    if not 'var1' in c:
        add_cols.append(c)

def calc_error(val_res):
    best_days = [i for i in np.argmin(np.array(val_res), axis=0)]
    return np.mean([np.array(val_res)[j,i] for i, j in enumerate(best_days)])

def add_func_feature(train_X, val_X, test_X, func, look_col):
    #15,5k
    train_X['var1_'+func.__name__] = train_X[look_col].apply(lambda x : func(x), axis=1)
    val_X['var1_'+func.__name__] = val_X[look_col].apply(lambda x : func(x), axis=1)
    test_X['var1_'+func.__name__] = test_X[look_col].apply(lambda x : func(x), axis=1)
    return train_X, val_X, test_X

def add_func_norm_feature(train_X, val_X, test_X, func, look_col):
    #15k
    train_X['var1_'+func.__name__] = train_X[look_col].apply(lambda x : func(x), axis=1)
    max_var1 = max(train_X['var1_'+func.__name__])
    train_X['var1_'+func.__name__] = train_X['var1_'+func.__name__]/max_var1
    
    val_X['var1_'+func.__name__] = val_X[look_col].apply(lambda x : func(x), axis=1)/max_var1
    test_X['var1_'+func.__name__] = test_X[look_col].apply(lambda x : func(x), axis=1)/max_var1
    return train_X, val_X, test_X

def add_pct_change_single_df(df, look_col, look_num, pct_col):
    temp = df[look_col[-look_num:]].pct_change()
    temp.columns =  pct_col
    temp = temp.fillna(0)
    temp.index=df.index
    df=pd.concat([df, temp], axis=1)
    return df

def add_pct_change(train_X, val_X, test_X, look_col, look_num=pred_size):
    pct_col =['var1_pct(t-{})'.format(i) for i in range(look_num, 0, -1)]

    train_X = add_pct_change_single_df(train_X, look_col, look_num, pct_col)
    val_X = add_pct_change_single_df(val_X, look_col, look_num, pct_col)
    test_X = add_pct_change_single_df(test_X, look_col, look_num, pct_col)
    return train_X, val_X, test_X

In [570]:
X_test.head()

,colour,rareness,revision,var1(t-43),var1(t-42),var1(t-41),var1(t-40),var1(t-39),var1(t-38),var1(t-37),...,var1(t-10),var1(t-9),var1(t-8),var1(t-7),var1(t-6),var1(t-5),var1(t-4),var1(t-3),var1(t-2),var1(t-1)
50,0,0,0,14.962763,14.962954,14.954980,14.975891,14.922806,14.930123,14.925979,...,15.012953,15.012056,15.000089,14.982642,15.013320,14.987340,14.960579,14.958263,14.993840,14.958823
50,0,0,1,13.562132,13.562392,13.573333,13.556119,13.547468,13.529041,13.492096,...,13.579617,13.576047,13.566062,13.572917,13.556965,13.543082,13.538018,13.525057,13.516720,13.517797
50,0,0,1,12.854575,12.870447,12.892741,12.892233,12.863090,12.827581,12.829240,...,12.927807,12.906334,12.916092,12.909462,12.894067,12.875680,12.862898,12.847195,12.844449,12.843990
50,0,0,0,13.323624,13.263196,13.249560,13.259117,13.252407,13.252017,13.302519,...,13.218886,13.229354,13.226208,13.228437,13.257499,13.193953,13.149173,13.177401,13.217206,13.220722
50,0,0,0,12.837897,12.827659,12.839326,12.857408,12.888437,12.859975,12.852136,...,12.867902,12.865782,12.834492,12.856785,12.871237,12.802158,12.755947,12.763935,12.778585,12.769228


In [581]:
val_res=[]
best_param=[]
res_dict ={}
predict_df = pd.DataFrame()
max_look_back = len(train_col)
for look_back in tqdm.tqdm(range(max_look_back, pred_size, -1)): #1
    look_col = train_col[-look_back:]
    train_X = X[add_cols+look_col]#add_cols+
    val_X = X_val[add_cols+look_col]
#     print(test_X.head(), val_X.head())
    test_X = X_test[add_cols+look_col]
#     print(look_back, look_col)



    train_X, val_X, test_X = add_func_norm_feature(train_X, val_X, test_X, np.max, look_col)
    train_X, val_X, test_X = add_func_norm_feature(train_X, val_X, test_X, np.min, look_col)
    train_X, val_X, test_X = add_func_norm_feature(train_X, val_X, test_X, np.std, look_col)
    train_X, val_X, test_X = add_func_norm_feature(train_X, val_X, test_X, np.mean, look_col)
    
    
    train_X, val_X, test_X = add_pct_change(train_X, val_X, test_X,look_col, 1)
    
    
#     print(train_X.head())
#     temp = train_X.groupby(['is_cheap'])[look_col[-pred_size:]].mean().reset_index()
#     print(train_X.shape)
    
#     train_X, val_X, test_X = add_applied_feature(train_X, val_X, test_X, skew, look_col)
#     train_X, val_X, test_X = add_applied_feature(train_X, val_X, test_X, kurtosis, look_col)
#     train_X, val_X, test_X = add_applied_feature(train_X, val_X, test_X, hmean, look_col)
#     train_X, val_X, test_X = add_applied_feature(train_X, val_X, test_X, gmean, look_col)


    val_res.append([])
    for day in range(len(test_col)):
        train_y = y[test_col[day]]
        val_y = y_val[test_col[day]]
        model = LinearRegression(n_jobs=-1, normalize=True)
        model.fit(train_X,train_y)
        val_yhat = model.predict(val_X)
        val_res[-1].append(evaluate(val_y, val_yhat))

        model.fit(val_X,val_y)
        test_yhat_day = model.predict(test_X)
        predict_df['lookback_{}_day_{}'.format(look_back, day)] = test_yhat_day

  3%|▎         | 1/36 [00:01<00:47,  1.36s/it]

['var1_pct(t-1)']


  6%|▌         | 2/36 [00:02<00:46,  1.36s/it]

['var1_pct(t-1)']


  8%|▊         | 3/36 [00:04<00:46,  1.41s/it]

...

['var1_pct(t-1)']


 97%|█████████▋| 35/36 [01:05<00:01,  1.90s/it]

['var1_pct(t-1)']


100%|██████████| 36/36 [01:07<00:00,  1.90s/it]

['var1_pct(t-1)']


In [582]:
best_lookbacks = [i for i in np.argmin(np.array(val_res), axis=0)]
best_val_score = np.mean([np.array(val_res)[j,i] for i, j in enumerate(best_lookbacks)])
print(best_val_score)
#15018

14134.5313693


In [527]:
best_lookbacks
#0 - 42 lookback

[34, 1, 11, 6, 10, 9, 9]

In [468]:
predict_df.shape

(1253, 253)

In [583]:
predict_df['player_id'] = sorted(data_provider.sample_subm.player_id.unique())
best_pred_columns =['lookback_{}_day_{}'.format(max_look_back - look_b, i) for i, look_b in enumerate(best_lookbacks) ]
final_df = pd.melt(predict_df, id_vars="player_id", value_vars=best_pred_columns)

In [457]:
data_provider.sample_subm.shape

(8771, 4)

In [584]:
d = sorted(data_provider.sample_subm.Date.unique().astype(str))
dates = {}
sample_subm = data_provider.sample_subm
for col, d in zip(best_pred_columns, d):
    dates[col] = d
final_df["variable"] = final_df["variable"].map(dates)
final_df.sort_values(["player_id", "variable"], inplace=True)
final_df.reset_index(drop=True, inplace=True)
sample_subm["price"] = final_df["value"]
sample_subm["price"] = np.exp(sample_subm["price"])
sample_subm.head()

,id,price,player_id,Date
0,1_2017-06-23,3.152877e+06,1,2017-06-23
1,1_2017-06-24,3.074659e+06,1,2017-06-24
2,1_2017-06-25,2.879574e+06,1,2017-06-25
3,1_2017-06-26,2.870236e+06,1,2017-06-26
4,1_2017-06-27,2.885394e+06,1,2017-06-27


In [585]:
sample_subm[["id", "price"]].to_csv('{}_linear_subm.csv'.format(int(best_val_score), index=False)